In [1]:
import requests
import pandas as pd
import json
import numpy as np

Définition des départements et des sports étudiés.

In [2]:
list_dept = [str(i).zfill(2) for i in range(1, 96) if i != 20]
list_dept.extend(["971", "972", "973", "974"])

list_sport = ['22Terrain%20de%20football%22', #Football (0)
              '22Court%20de%20tennis%22', #Tennis (1)
              '22Salle%20multisports%20(gymnase)%22', #Gymnase (2)
              '22Salle%20de%20basket%22', #salle basket (3)
              '22Terrain%20de%20basket-ball%22', #terrain basket (4)
              '22Dojo%20%2F%20Salle%20d%27arts%20martiaux%22', #Arts martiaux (judo) (5)
              '22Terrain%20de%20handball%22', #Terrain Handball (6)
              '22Salle%20de%20handball%22', #Salle Handball (7)
              '22Parcours%2018%20trous%22', #Parcours 18 trous (golf) (8)
              '22Parcours%209%20trous%22', #Parcours 9 trous (glof) (9)
              '22Practice%22', #Practice (golf) (10)
              '22Bassin%20sportif%20de%20natation%22', #Bassin sportif de natation (11)
              '22Bassin%20ludique%20de%20natation%22', #Bassin ludique de natation (12)
              '22Bassin%20mixte%20de%20natation%22', #Bassin mixte de natation (13)
              '22Terrain%20de%20rugby%22', #Terrain de rugby (14)
              ]

## Récupération du nombre d'infrastructure, par sport et pas département.

In [3]:
sports_infrastructures = []

# Iterate over each sport in the list_sport
for sport in list_sport:
    sport_infra = []
    
    # Iterate over each department in the list_dept
    for dept in list_dept:
        try:
            # Construct the URL for API request
            url = f"https://equipements.sports.gouv.fr/api/explore/v2.1/catalog/datasets/data-es/records?limit=100&offset=0&refine=carac159%3A%22Clubs%20sportifs%2C%20comités%2C%20ligues%2C%20fédérations%22&refine=typequipement%3A%{sport}&refine=dep_code_filled%3A%22{dept}%22"

            # Fetch data from the API
            response = requests.get(url)
            data_json = response.json()

            # Extract the total count of infrastructures
            nb_infra = data_json["total_count"]
            
            # Append the count to the list
            sport_infra.append(nb_infra)
        except Exception as e:
            # Handle exceptions gracefully
            print(f"Error fetching data for {sport} in department {dept}: {e}")
            sport_infra.append(None)  # Append None if an error occurs

    # Append the list of infrastructure counts for the current sport to the main list
    sports_infrastructures.append(sport_infra)


396
332
311
59
28
152
152
226
55
203
111
215
374
398
152
286
348
218
151
299
392
109
251
356
201
320
227
709
320
463
111
644
257
700
274
319
524
207
127
220
397
184
598
329
137
123
53
659
341
372
186
394
392
97
627
699
151
1103
470
197
924
338
246
103
68
711
450
438
171
432
460
147
256
68
570
434
363
354
355
244
173
159
200
462
381
254
194
259
82
326
153
188
189
242
159
133
69
198
437
205
209
119
103
461
168
93
87
178
228
155
841
422
75
310
454
196
113
330
112
47
257
248
297
236
208
191
460
705
170
969
727
228
150
277
734
110
340
234
383
145
516
358
103
226
46
427
249
285
122
105
342
68
250
609
134
759
399
138
352
279
308
157
249
718
423
746
84
263
274
326
509
398
534
747
884
195
257
173
129
466
321
403
284
164
197
237
80
749
658
329
420
507
95
88
43
173
155
116
96
24
32
125
60
118
31
71
55
88
289
176
40
102
147
64
64
107
136
51
102
139
105
117
89
336
105
264
53
267
148
398
66
138
320
74
109
92
219
67
453
182
44
113
18
345
130
149
44
107
173
55
248
272
53
742
204
76
396
152
143
57
79
3

In [24]:
print(len(sports_infrastructures))
print(len(sports_infrastructures[1]))

15
98


On regroupe ensuite les types d'infrastructures par sport. Exemple : la natation regroupe les bassins sportifs, ludique et mixte; le golf regroupe les parcours 18 trous, 9 trous et les practice; et les gymnases multisport contribuent à la fois au basketball et au handball.

In [21]:
Football_infra = np.array(sports_infrastructures[0])
Tennis_infra = np.array(sports_infrastructures[1])
Basket_infra = np.array(sports_infrastructures[2]) + np.array(sports_infrastructures[3]) #terrain basket + gymnase
Judo_infra = np.array(sports_infrastructures[5])
Handball_infra = np.array(sports_infrastructures[6]) + np.array(sports_infrastructures[7]) + + np.array(sports_infrastructures[2]) #terrain hand + salle hand + gymnase
Golf_infra = np.array(sports_infrastructures[8]) + np.array(sports_infrastructures[9]) + + np.array(sports_infrastructures[10]) #18 trous + 9 trous + practice                                                       
Natation_infra = np.array(sports_infrastructures[11]) + np.array(sports_infrastructures[12]) + + np.array(sports_infrastructures[13]) #Bassin sportif + ludique + mixte 
Rugby_infra = np.array(sports_infrastructures[14])

## Récupération de l'âge moyen des infrastructures par sport et par département.

In [22]:
# Initialize an empty list to store age information for sports infrastructures
Ages_sport_infrastructures = []

# Iterate over each sport in the list_sport
for sport in list_sport:
    # Initialize an empty list to store age information for the current sport
    Age_sport_infra = []
    
    # Iterate over each department in the list_dept
    for dept in list_dept:
        # Fetch the total count of records for the current sport and department
        total_count = json.loads(requests.get('https://equipements.sports.gouv.fr/api/explore/v2.1/catalog/datasets/data-es/records?limit=0&offset=0&refine=carac159%3A%22Clubs%20sportifs%2C%20comités%2C%20ligues%2C%20fédérations%22&refine=typequipement%3A%' + sport + '&refine=dep_code_filled%3A%22' + dept + '%22').text)["total_count"]
        
        # Initialize variables for calculating the average age
        sum = 0
        i = 0

        batch_size = 100
        start_index = 0

        # Iterate over batches of data until all records are processed
        while start_index < total_count:
            # Check if there are fewer than 100 elements left for the last batch
            if total_count - start_index < batch_size:
                batch_size = total_count - start_index
            
            # Fetch data for the current batch
            data = json.loads(requests.get('https://equipements.sports.gouv.fr/api/explore/v2.1/catalog/datasets/data-es/records?select=carac118&limit=' + str(batch_size) + '&offset=' + str(start_index) + '&refine=carac159%3A%22Clubs%20sportifs%2C%20comités%2C%20ligues%2C%20fédérations%22&refine=typequipement%3A%' + sport + '&refine=dep_code_filled%3A%22' + dept + '%22').text)["results"]
            
            # Iterate over each record in the batch
            for elt in data:
                # Check if the age information is a string and is numeric
                if isinstance(elt["carac118"], str) and elt["carac118"].isdigit():
                    # Update sum and count
                    sum += int(elt["carac118"])
                    i += 1
            
            # Update the start index for the next iteration
            start_index += batch_size

        # Calculate the average age if there are records available
        if total_count != 0 and i != 0:
            Age_sport_infra.append(2024 - round(sum/i))  # Append the average age to the list
        else:
            Age_sport_infra.append(None)  # Append None if there are no records
        
        # Append the list of average ages for the current sport and department to the main list
    Ages_sport_infrastructures.append(Age_sport_infra)


22Terrain%20de%20football%22 01 40
22Terrain%20de%20football%22 02 41
22Terrain%20de%20football%22 03 45
22Terrain%20de%20football%22 04 33
22Terrain%20de%20football%22 05 47
22Terrain%20de%20football%22 06 32
22Terrain%20de%20football%22 07 40
22Terrain%20de%20football%22 08 38
22Terrain%20de%20football%22 09 43
22Terrain%20de%20football%22 10 40
22Terrain%20de%20football%22 11 32
22Terrain%20de%20football%22 12 44
22Terrain%20de%20football%22 13 40
22Terrain%20de%20football%22 14 41
22Terrain%20de%20football%22 15 40
22Terrain%20de%20football%22 16 46
22Terrain%20de%20football%22 17 43
22Terrain%20de%20football%22 18 41
22Terrain%20de%20football%22 19 43
22Terrain%20de%20football%22 21 44
22Terrain%20de%20football%22 22 35
22Terrain%20de%20football%22 23 47
22Terrain%20de%20football%22 24 45
22Terrain%20de%20football%22 25 40
22Terrain%20de%20football%22 26 43
22Terrain%20de%20football%22 27 38
22Terrain%20de%20football%22 28 42
22Terrain%20de%20football%22 29 33
22Terrain%20de%20foo

In [25]:
print(len(Ages_sport_infrastructures))
print(len(Ages_sport_infrastructures[1]))

30
98


On remplace les None par des valeurs numériques 0.

In [52]:
Ages_sport_infrastructures2 = Ages_sport_infrastructures[::2]

for liste in Ages_sport_infrastructures2:
    for i in range(len(liste)):
        if liste[i] is None:
            liste[i] = 0

Ages_sport_infrastructures2 = np.array(Ages_sport_infrastructures2)

On regroupe comme précédemment les types d'infrastructures par sport.

In [53]:
Football_age = np.array(Ages_sport_infrastructures2[0])
Tennis_age = np.array(Ages_sport_infrastructures2[1])
Basket_age = np.array(Ages_sport_infrastructures2[2]) + np.array(Ages_sport_infrastructures2[3]) #terrain basket + gymnase
Judo_age = np.array(Ages_sport_infrastructures2[5])
Handball_age = np.array(Ages_sport_infrastructures2[6]) + np.array(Ages_sport_infrastructures2[7]) + np.array(Ages_sport_infrastructures2[2]) #terrain hand + salle hand + gymnase
Golf_age = np.array(Ages_sport_infrastructures2[8]) + np.array(Ages_sport_infrastructures2[9]) + np.array(Ages_sport_infrastructures2[10]) #18 trous + 9 trous + practice                                                       
Natation_age = np.array(Ages_sport_infrastructures2[11]) + np.array(Ages_sport_infrastructures2[12]) + np.array(Ages_sport_infrastructures2[13]) #Bassin sportif + ludique + mixte 
Rugby_age = np.array(Ages_sport_infrastructures2[14])

On exporte les données récupérées en CSV.

In [57]:
# Create a DataFrame with specified columns and data
df = pd.DataFrame({'code_insee' : list_dept,
                   'Infrastructures_Football' : Football_infra,
                   'Infrastructures_Tennis' : Tennis_infra,
                   'Infrastructures_Basketball' : Basket_infra,
                   'Infrastructures_Judo' : Judo_infra,
                   'Infrastructures_Handball' : Handball_infra,
                   'Infrastructures_Golf' : Golf_infra,
                   'Infrastructures_Natation' : Natation_infra,
                   'Infrastructures_Rugby' : Rugby_infra,

                   'AgeInfrastructures_Football' : Football_age,
                   'AgeInfrastructures_Tennis' : Tennis_age,
                   'AgeInfrastructures_Basketball' : Basket_age,
                   'AgeInfrastructures_Judo' : Judo_age,
                   'AgeInfrastructures_Handball' : Handball_age,
                   'AgeInfrastructures_Golf' : Golf_age,
                   'AgeInfrastructures_Natation' : Natation_age,
                   'AgeInfrastructures_Rugby' : Rugby_age
                   })

df.to_csv("/Users/titouanlegourrierec/Desktop/Sport_Equipment.csv")